In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
!git clone https://github.com/SeanNaren/warp-ctc.git

fatal: destination path 'warp-ctc' already exists and is not an empty directory.


In [4]:
%cd warp-ctc

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc


In [5]:
mkdir -p build

In [6]:
cd build

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/build


In [7]:
!cmake ..

-- cuda found TRUE
-- Building shared library with GPU support
-- Configuring done
-- Generating done
-- Build files have been written to: /workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/build


In [8]:
!make -j

[ 11%] Linking CXX shared library libwarpctc.so
[ 33%] Built target warpctc
[ 44%] Linking CXX executable test_cpu
[ 55%] Linking CXX executable test_gpu
[ 77%] Built target test_cpu
[100%] Built target test_gpu


In [9]:
cd ..

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc


In [10]:
cd pytorch_binding

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/pytorch_binding


In [11]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing warpctc_pytorch.egg-info/PKG-INFO
writing dependency_links to warpctc_pytorch.egg-info/dependency_links.txt
writing top-level names to warpctc_pytorch.egg-info/top_level.txt
writing manifest file 'warpctc_pytorch.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
building 'warpctc_pytorch._warp_ctc' extension
Emitting ninja build file /workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/pytorch_binding/build/temp.linux-x86_64-3.9/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
g++ -pthread -B /workspace/zhanghui/DeepSpeech-2.x/tools/venv/compiler_compat -Wl,--sysroot=/ -shared -Wl,-rpath,/workspace/zhanghui/DeepSpeech-2.x/tools/venv/lib -Wl,-rpath-link,/workspace/zhanghui/DeepSpeech-2.x/tools/venv/lib -L/workspace/zhang

In [12]:
!python -V

Python 3.9.5


In [13]:
cd ..

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import warpctc_pytorch as wp
import paddle.nn as pn
import paddle

In [15]:
torch.__version__

'1.10.0+cu102'

In [16]:
paddle.__version__

'2.2.1'

In [17]:
# warpctc_pytorch CTCLoss
probs = torch.FloatTensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]
    ]]).transpose(0, 1).contiguous()
print(probs.size())
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)
bs = probs.size(1)

ctc_loss = wp.CTCLoss(size_average=False, length_average=False)
cost = ctc_loss(probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

torch.Size([2, 1, 5])
2.4628584384918213
[[[ 0.17703122 -0.70812464  0.17703122  0.17703122  0.17703122]]

 [[ 0.17703122  0.17703122 -0.70812464  0.17703122  0.17703122]]]


In [18]:
# pytorch CTCLoss
probs = torch.FloatTensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]
    ]]).transpose(0, 1).contiguous()
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)
bs = probs.size(1)

log_probs = torch.log_softmax(probs, axis=-1)

ctc_loss1 = nn.CTCLoss(reduction='none')
cost = ctc_loss1(log_probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

2.4628584384918213
[[[ 0.1770312 -0.7081248  0.1770312  0.1770312  0.1770312]]

 [[ 0.1770312  0.1770312 -0.7081248  0.1770312  0.1770312]]]


In [27]:
# Paddle CTCLoss
paddle.set_device('cpu')
probs = paddle.to_tensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1],
    ]]).transpose([1,0,2])
print(probs.shape) # (T, B, D)
labels = paddle.to_tensor([[1, 2]], dtype='int32') #（B，L)
print(labels.shape)
label_sizes = paddle.to_tensor([2], dtype='int64')
probs_sizes = paddle.to_tensor([2], dtype='int64')
bs = paddle.shape(probs)[1]
probs.stop_gradient=False

ctc_loss = pn.CTCLoss(reduction='none')
cost = ctc_loss(probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

[2, 1, 5]
[1, 2]
2.4628584384918213
[[[ 0.17703122 -0.70812464  0.17703122  0.17703122  0.17703122]]

 [[ 0.17703122  0.17703122 -0.70812464  0.17703122  0.17703122]]]


In [26]:
# warpctc_pytorch CTCLoss, log_softmax idempotent
probs = torch.FloatTensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]
    ]]).transpose(0, 1).contiguous()
print(probs.size())
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)
bs = probs.size(1)

ctc_loss = wp.CTCLoss(size_average=False, length_average=False)

log_probs = torch.log_softmax(probs, axis=-1)
cost = ctc_loss(log_probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

torch.Size([2, 1, 5])
2.4628584384918213
[[[ 0.17703117 -0.7081247   0.17703117  0.17703117  0.17703117]]

 [[ 0.17703117  0.17703117 -0.7081247   0.17703117  0.17703117]]]
